In [72]:
#packages needed to do the project. 
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import requests
import os
import io

# Gather: 

### Data Sources: 
For this project, we will use three different data sources. 
    1. The WeRateDogs Twitter archive, which is provided by WeRateDog to Udacity via a csv file. 
    2. The Tweet Image prediction, which we will have to download programmatically using Python's requests library. 
    3. Each Tweet's retweet and favorite ('like') count, which we will download from Twitter using Tweepy a twitter API. 
    

##### Enhanced Twitter Archive:
- The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).

In [11]:

twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [13]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [14]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

##### Tweet Image prediction: 

We will use Python's request and io libraries to download this dataset from a Udacity hosted server. 

In [41]:

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url).content
image_pred = pd.read_csv(io.StringIO(response.decode('utf-8')), sep='\t')

In [42]:
#Save the file in a csv format. 
#image_pred.to_csv('image-predictions.csv', index=False)


In [43]:
image_pred.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
364,672884426393653248,https://pbs.twimg.com/media/CVaQ0M4UsAAki3t.jpg,1,tusker,0.122410,False,warthog,0.119870,False,water_buffalo,0.105856,False
1282,750383411068534784,https://pbs.twimg.com/media/CmnluwbXEAAqnkw.jpg,1,Border_collie,0.672791,True,collie,0.270188,True,papillon,0.034504,True
354,672591762242805761,https://pbs.twimg.com/media/CVWGotpXAAMRfGq.jpg,1,kuvasz,0.777659,True,Great_Pyrenees,0.112517,True,golden_retriever,0.038351,True
1656,810657578271330305,https://pbs.twimg.com/media/C0AIwgVXAAAc1Ig.jpg,1,malamute,0.753521,True,Siberian_husky,0.166151,True,Eskimo_dog,0.069811,True
755,688385280030670848,https://pbs.twimg.com/media/CY2iwGNWUAI5zWi.jpg,2,golden_retriever,0.900437,True,cocker_spaniel,0.022292,True,sombrero,0.014997,False


Additional Data via the Twitter API

- Back to the basic-ness of Twitter archives: retweet count and favorite count are two of the notable column omissions. Fortunately, this additional data can be gathered by anyone from Twitter's API. Well, "anyone" who has access to data for the 3000 most recent tweets, at least. But you, because you have the WeRateDogs Twitter archive and specifically the tweet IDs within it, can gather this data for all 5000+. And guess what? You're going to query Twitter's API to gather this valuable data.

In [65]:
#Consumer Key and Scecret Codes: 
consumer_key = 'sIxqqO3GYMtb19GPfnYT9m1jJ'
consumer_secret = 'F6tUkGIhSXVyLtjyi9zdtbTQ8IbEbaFAHUb4Mvg68leQYKALRO'
access_token= '1044734987319205888-wBxtFZXiXTcyDx2WkON4rB5TapfdoY'
access_token_secret = 'tNZPiIsHXoe9LVwdOg4u8SGY61WoqkPg1VOZcjWYvnQLx'

In [66]:
#Authenticating my twitter account through tweepy. 
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [67]:
#to check if our API is working: 
user = api.me()
print (user.name)

nfaheem


In [70]:
for status in tweepy.Cursor(api.home_timeline).items(1):
    # Process a single status
    print(status.text)

The IoT-Connected Car of Today Cases From Hertz, Nokia, NTT, Mojio &amp; Concur Technologies https://t.co/CwpQ501kSH


In [101]:
#We have to feed the line below a tweet id 
tweet = api.get_status('892177421306343426',tweet_mode='extended')
#getting the retweet_count for the user id: 
tweet._json['retweet_count']
#getting the favorite ('like') count for each tweet. 
tweet._json['favorite_count']

The way I am going to handle this is as follows: 
1. create a list of the twiter ID so that I could loop over it and get the retweet_count and favorite_count. 
2. append the retweet_count and favorite_count to the new list along the tweet id. 


In [163]:
tweet._json['entities']['media'][0]['media_url']

'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg'

In [159]:
new_json[0]['media_url']

'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg'

In [ ]:
'''
#This is used to creat authorization: 

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print tweet.text
    
'''

In [ ]:
#Get retweet count and favorite count for each tween in our df dataframe. 

In [ ]:
list_id = (twitter_archive['tweet_id'])
twitter_dict = [] 
for id in list_id:
    twitter_dict = [] 
    try:
        tweet = api.get_status(id, tweet_mode='extended') 
        twitter_archive.append(tweet) 
    except: twitter_archive.append('Tweet deleted')
            

Image Predictions File

# Assess: 

# Clean